In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LinearRegression

# Veriyi okuma ve işleme
index_isimleri = ['birim_no', 'zaman_döngüleri']
ayar_isimleri = ['ayar_1', 'ayar_2', 'ayar_3']
sensör_isimleri = ['sensör_{}'.format(i) for i in range(1, 22)]
kolon_isimleri = index_isimleri + ayar_isimleri + sensör_isimleri

egitim = pd.read_csv(r"D:\users\m_ozdemir15\Desktop\Okul\Y.Lisans\2.Dönem\Makine Öğrenmesi\Proje\cmapss\train_FD001.txt", sep='\s+', header=None, names=kolon_isimleri)
test = pd.read_csv(r"D:\users\m_ozdemir15\Desktop\Okul\Y.Lisans\2.Dönem\Makine Öğrenmesi\Proje\cmapss\test_FD001.txt", sep='\s+', header=None, names=kolon_isimleri)
y_test = pd.read_csv(r"D:\users\m_ozdemir15\Desktop\Okul\Y.Lisans\2.Dönem\Makine Öğrenmesi\Proje\cmapss\RUL_FD001.txt", sep='\s+', header=None, names=['RUL'])

def geriye_kalan_kullanılabilir_ömür_ekle(df):
    # Boş bir DataFrame oluştur
    sonuç_cercevesi = pd.DataFrame()

    # Her bir birim (unit) için RUL hesaplamalarını yap
    for birim_no, grup in df.groupby("birim_no"):
        maksimum_döngü = grup["zaman_döngüleri"].max()
        grup["RUL"] = maksimum_döngü - grup["zaman_döngüleri"]
        sonuç_cercevesi = pd.concat([sonuç_cercevesi, grup])

    return sonuç_cercevesi

# Veriyi hazırla
egitim = geriye_kalan_kullanılabilir_ömür_ekle(egitim)

atılacak_kolonlar = index_isimleri + ayar_isimleri
# Sadece sensör verileri dışındaki sütunları bırak
X_egitim = egitim.drop(atılacak_kolonlar, axis=1)
y_egitim = X_egitim.pop('RUL')

# Test verileri için RUL değerleri sadece her motorun son zaman döngüsünde verildiği için test verilerini ayarla
X_test = test.groupby('birim_no').last().reset_index().drop(atılacak_kolonlar, axis=1)

# Veri bölünmelerini oluştur
X_egitim, X_doğrulama, y_egitim, y_doğrulama = train_test_split(X_egitim, y_egitim, test_size=0.2)

# Modeli oluşturun
model = LinearRegression()

# Modeli eğitin
model.fit(X_egitim, y_egitim)

# Eğitim verileri üzerinde modelin performansını değerlendirin
y_egitim_tahmini = model.predict(X_egitim)

# Doğrulama verileri üzerinde modelin performansını değerlendirin
y_doğrulama_tahmini = model.predict(X_doğrulama)

# Eğitim verileri üzerinde hata metriklerini hesaplayın
egitim_başarı_oranı = model.score(X_egitim, y_egitim)

# Doğrulama verileri üzerinde hata metriklerini hesaplayın
doğrulama_başarı_oranı = model.score(X_doğrulama, y_doğrulama)

# Hata metriklerini yazdırın
print(f"Eğitim R^2 Skoru: {egitim_başarı_oranı}")
print(f"Doğrulama R^2 Skoru: {doğrulama_başarı_oranı}")

# Eşik değerini tanımla
eşik_değeri = 50

# RUL değerlerini sınıflandırın
y_egitim_sınıflandırma = (y_egitim <= eşik_değeri).astype(int)
y_doğrulama_sınıflandırma = (y_doğrulama <= eşik_değeri).astype(int)

# Doğrulama verileri üzerinde doğruluk skorunu ve sınıflandırma raporunu hesaplayın
doğruluk = accuracy_score(y_doğrulama_sınıflandırma, model.predict(X_doğrulama) <= eşik_değeri)
rapor = classification_report(y_doğrulama_sınıflandırma, model.predict(X_doğrulama) <= eşik_değeri)

print(f"Doğruluk Skoru: {doğruluk}")
print("Sınıflandırma Raporu:")
print(rapor)


Eğitim R^2 Skoru: 0.5796507800820856
Doğrulama R^2 Skoru: 0.578960530425569
Doğruluk Skoru: 0.9023503755754786
Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      3122
           1       0.99      0.61      0.75      1005

    accuracy                           0.90      4127
   macro avg       0.94      0.80      0.85      4127
weighted avg       0.91      0.90      0.89      4127

